### ✨ Captcha de Cloudflare

Turnstile es un captcha de Cloudflare que evita que los bots accedan a un sitio web.

**Por qué se utiliza:**

Cloudflare Turnstile se emplea para proteger sitios web de diversas amenazas automatizadas, incluyendo bots, spam y tráfico malicioso, sin comprometer la experiencia del usuario. Los CAPTCHA tradicionales a menudo requieren que los usuarios resuelvan acertijos, lo que puede ser frustrante y consumir mucho tiempo. Turnstile tiene como objetivo proporcionar una capa de seguridad sin fricciones, asegurando que solo los usuarios humanos legítimos puedan acceder a los recursos del sitio web. Esto ayuda a prevenir actividades como el relleno de credenciales (credential stuffing), ataques DDoS, el raspado de contenido (content scraping) y otras formas de abuso que dependen de scripts automatizados.

**Cómo funciona:**
Turnstile opera ejecutando una serie de desafíos no intrusivos del lado del cliente en segundo plano. Cuando un usuario visita una página protegida por Turnstile, se carga un pequeño widget de JavaScript. Este widget luego realiza varias verificaciones y análisis del entorno y comportamiento del navegador del usuario. Estas verificaciones pueden incluir:

1.  **Prueba de Trabajo (Proof of Work):** Un pequeño desafío computacional que es fácil para el dispositivo de un humano pero intensivo en recursos para una gran botnet.
2.  **Análisis de API del Navegador:** Examinar las características y capacidades del navegador que son comunes en usuarios humanos pero a menudo ausentes o manipuladas en entornos de bots.
3.  **Modelos de Aprendizaje Automático (Machine Learning Models):** Analizar patrones de comportamiento, huellas dactilares de dispositivos (device fingerprints) y características de red para distinguir entre interacciones humanas y automatizadas.

Crucialmente, la mayoría de estas verificaciones no son interactivas, lo que significa que el usuario no tiene que hacer clic en imágenes, escribir texto o resolver acertijos. Turnstile aprovecha la vasta inteligencia de red de Cloudflare para identificar y mitigar amenazas. Si el sistema detecta actividad sospechosa, podría escalar a un desafío ligeramente más complejo, pero generalmente aún no interactivo. El objetivo final es verificar la humanidad de forma fluida, permitiendo que los usuarios legítimos procedan sin interrupción mientras bloquea eficazmente los bots maliciosos.


In [3]:
%pip install requests anticaptchaofficial beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import requests
from bs4 import BeautifulSoup
from anticaptchaofficial.turnstileproxyless import turnstileProxyless
import json

PAGE_URL = "https://muisca.dian.gov.co/WebRutMuisca/DefConsultaEstadoRUT.faces"
BASE_PAGE_URL = "https://muisca.dian.gov.co"

# cloudflare turnstile site key
MUISCA_DIAN_SITE_KEY = "0x4AAAAAAAg1YFKr1lxPdUIL"
API_KEY_ANTICAPTCHA = ""

In [10]:
def get_rut_verification_result():
    session = requests.Session()
    response = session.get(
        f"{BASE_PAGE_URL}/WebRutMuisca/DefConsultaEstadoRUT.faces",
    )

    if response.status_code != 200:
        raise Exception(
            f"Failed to load DIAN RUT verification page, "
            f"status code: {response.status_code}"
        )

    soup = BeautifulSoup(response.text, "html.parser")

    view = soup.find(
        "input",
        {"name": "com.sun.faces.VIEW"},
    )
    query = soup.find(
        "input",
        {"name": "vistaConsultaEstadoRUT:formConsultaEstadoRUT"},
    )

    if None in [view, query]:
        raise Exception(
            "Failed to extract verification data from DIAN login page. "
            "Check that the page to look up the RUT is working."
        )

    return {
        "view": view["value"],
        "query": query["value"],
        "btn_x": 29,
        "btn_y": 7,
        "cookies": session.cookies.get_dict(),
    }


def resolve_cloudflare_turnstile(site_key: str, url: str) -> str:
    solver = turnstileProxyless()
    solver.set_verbose(1)
    solver.set_key(API_KEY_ANTICAPTCHA)
    solver.set_website_url(url)
    solver.set_website_key(site_key)
    solver.set_soft_id(0)

    token = solver.solve_and_return_solution()

    if not token:
        raise Exception("Cloudflare Turnstile solving failed.")

    return token

In [ ]:
def _is_user(soup):
    first_name_element = soup.find(
        "span",
        {"id": "vistaConsultaEstadoRUT:formConsultaEstadoRUT:primerNombre"},
    )
    return bool(
        first_name_element
        and first_name_element.text.strip()
        and first_name_element.text.strip() != "N/A"
    )


def _get_company_info(soup):
    company_name = soup.find(
        "span",
        {"id": "vistaConsultaEstadoRUT:formConsultaEstadoRUT:razonSocial"},
    )
    company_name_value = company_name.text if company_name else "N/A"

    return {
        "company_name": company_name_value,
    }


def _get_user_info(soup):
    last_name_first = soup.find(
        "span",
        {"id": "vistaConsultaEstadoRUT:formConsultaEstadoRUT:primerApellido"},
    )
    last_name_first_value = last_name_first.text if last_name_first else "N/A"

    last_name_second = soup.find(
        "span",
        {"id": "vistaConsultaEstadoRUT:formConsultaEstadoRUT:segundoApellido"},
    )
    last_name_second_value = last_name_second.text if last_name_second else "N/A"

    first_name = soup.find(
        "span",
        {"id": "vistaConsultaEstadoRUT:formConsultaEstadoRUT:primerNombre"},
    )
    first_name_value = first_name.text if first_name else "N/A"

    additional_names = soup.find(
        "span",
        {"id": "vistaConsultaEstadoRUT:formConsultaEstadoRUT:otrosNombres"},
    )
    additional_names_value = additional_names.text if additional_names else "N/A"

    return {
        "last_name_first": last_name_first_value,
        "last_name_second": last_name_second_value,
        "first_name": first_name_value,
        "additional_names": additional_names_value,
    }


def get_rut_status(
    nit,
    verification_result,
):
    if not verification_result["cf_turnstile_token"]:
        raise Exception("Invalid Cloudflare Turnstile response token.")

    session = requests.Session()
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
    }
    payload = {
        "com.sun.faces.VIEW": verification_result["view"],
        "vistaConsultaEstadoRUT:formConsultaEstadoRUT": verification_result["query"],
        "vistaConsultaEstadoRUT:formConsultaEstadoRUT:_idcl": "",
        "vistaConsultaEstadoRUT:formConsultaEstadoRUT:btnBuscar.x": verification_result[
            "btn_x"
        ],
        "vistaConsultaEstadoRUT:formConsultaEstadoRUT:btnBuscar.y": verification_result[
            "btn_y"
        ],
        "vistaConsultaEstadoRUT:formConsultaEstadoRUT:hddToken": verification_result[
            "cf_turnstile_token"
        ],
        "vistaConsultaEstadoRUT:formConsultaEstadoRUT:hddllavePublica": MUISCA_DIAN_SITE_KEY,
        "vistaConsultaEstadoRUT:formConsultaEstadoRUT:mantenerCriterios": "",
        "vistaConsultaEstadoRUT:formConsultaEstadoRUT:modoOperacionFormBO": "",
        "vistaConsultaEstadoRUT:formConsultaEstadoRUT:modoPresentacionFormBO": "pantalla",
        "vistaConsultaEstadoRUT:formConsultaEstadoRUT:modoPresentacionSeleccionBO": "pantalla",
        "vistaConsultaEstadoRUT:formConsultaEstadoRUT:numNit": nit,
        "vistaConsultaEstadoRUT:formConsultaEstadoRUT:siguienteURL": "",
        "cf-turnstile-response": verification_result["cf_turnstile_token"],
    }

    for key, value in verification_result["cookies"].items():
        session.cookies.set(key, value)

    response = session.post(
        f"{BASE_PAGE_URL}/WebRutMuisca/DefConsultaEstadoRUT.faces",
        data=payload,
        headers=headers,
    )

    if response.status_code != 200:
        raise Exception(
            f"Failed to get DIAN RUT status page, status code: {response.status_code}"
        )

    soup = BeautifulSoup(response.text, "html.parser")

    has_status = soup.find(
        "td",
        {"class": "fondoTituloLeftAjustado"},
        string=lambda text: text and "Estado" in text,
    )
    if not has_status:
        raise Exception("Failed to extract RUT status from DIAN response page.")

    nit = soup.find(
        "input",
        {"id": "vistaConsultaEstadoRUT:formConsultaEstadoRUT:numNit"},
    )
    status = soup.find(
        "span",
        {"id": "vistaConsultaEstadoRUT:formConsultaEstadoRUT:estado"},
    )
    dv = soup.find("span", {"id": "vistaConsultaEstadoRUT:formConsultaEstadoRUT:dv"})

    if None in [nit, status, dv]:
        raise Exception("Failed to extract RUT info from DIAN response page.")

    rut_type = "user" if _is_user(soup) else "company"
    user_info = _get_user_info(soup) if rut_type == "user" else None
    company_info = _get_company_info(soup) if rut_type == "company" else None

    return {
        "nit": nit["value"],
        "dv": dv.text.strip(),
        "status": status.text.strip(),
        "type": rut_type,
        "user": user_info,
        "company": company_info,
        "updated_ago_in_minutes": 0,
    }

In [ ]:
verification_result = get_rut_verification_result()
print("✨ Verification result: ", json.dumps(verification_result, indent=4))

token = resolve_cloudflare_turnstile(
    url=f"{BASE_PAGE_URL}/WebRutMuisca/DefConsultaEstadoRUT.faces",
    site_key=MUISCA_DIAN_SITE_KEY,
)

verification_result["cf_turnstile_token"] = token
print("✨ Token: ", token)

status = get_rut_status(
    "0000",
    verification_result,
)
print("✨ Status: ", json.dumps(status, indent=4))

✨ Verification result:  {
    "view": "H4sIAAAAAAAAAO1cXWwcR3KeXZKSSMmydbrTnS3LbkmWaCHicrn8Ef99FCmaPPBHR1HynR1n3TvTuxx5dnrU07Ncyvbh7h4SBHm4AEmAJHCQGMjjOX9O8h7oIUCAC5AD8pKn5BLgEuQPyUt+Xi5dPT87szu7y1kNc7KgfdidnZ/qqq+qq6qru+f7/6oM2EyZpaySw/dxfeK+Xc5hyzJ0FXOdmrklcfIOx5xsYhNXCHttlxFyhzOHO4xsUY1893v//uuPypcHTylK3XqoKP2KcjlCTaVVi5rE5JLWPZ3s71DKlcFizT90Pxll+D6u4XqujFVihx67u76Fq7pZWaYmx7pJmHKuptsci/+2Y3B8SxxrdOfurkfomHJTpbkKreU0HZu5qqPbKs5h9sDROVEF3zi3T0q5CG+iwZU1XjVuY5MYbzJdU/qLupZX/E9aFAsBxVs9U9x2uOXwVcqqmEuiE0dBdDIgutAzUSCnfKksvtvp6qSy0jP1dVNwvKZrGjGV81Wq0duM2OI6VoXh3iEGUeHg5nYgSDpNnbL1iqPDxbs7GynTPtcsBiCYugRnoZVti7CjaUI6AWjnTBWLq6LDLjPxNNOpnbIgJ/Y0bZe+L47Spfu8oGsYuEZuOyXwhAH5pZ7JbxLTxveJLfvWVKp+ZQg6mGNgAbBHtT81f3UjcM7Lj+ladknddSzTqTvVGY/eQCoUBwTFsXyAZCpyS5pjKQgurVSSPGY61S2dBzTT47OQBk3pBBqEs1otVbVLRsc9gtn0SPqhtC9FPCePQEdTR0DzxhHQTLezS5J+bx9Kj81CPl3RTzL8UKQgVNWxkT6ohbF0aZ62mF4lbMkihqFrNEXi0gn4XKfiVhpcP2+TimNq9AjY9jkeT5fmKRfnLVotMXIE7E6kzC7ljNout43ELa1uXPC9YooRtnAEXrFwBF6xMJ0uzWNEjqrS15Hvah9LR